# Enade data transformation
Preliminary understanding of the data previously gathered for future transformations.
Performing some preliminary exploratory evaluations

In [1]:
# Import the libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Editing the csv file before reading it (changing decimal and sep to prevent bugs)
text = open("./enade2019/3.DADOS/microdados_enade_2019.txt", "r")
text = ''.join([i for i in text]) \
    .replace(",", ".") \
    .replace('; ',';')  
x = open("./enade2019/3.DADOS/microdados_enade_2019_2.txt","w")
x.writelines(text)
x.close()

In [3]:
# Reading with pandas
enade = pd.read_csv("./enade2019/3.DADOS/microdados_enade_2019_2.txt",
                    sep = ";",
                    decimal = ".",
                    low_memory=False)

In [4]:
enade.head()

,NU_ANO,CO_IES,CO_CATEGAD,CO_ORGACAD,CO_GRUPO,CO_CURSO,CO_MODALIDADE,CO_MUNIC_CURSO,CO_UF_CURSO,CO_REGIAO_CURSO,...,QE_I59,QE_I60,QE_I61,QE_I62,QE_I63,QE_I64,QE_I65,QE_I66,QE_I67,QE_I68
0,2019,1,10002,10028,5710,3,1,5103403,51,5,...,2.0,5.0,1.0,1.0,2.0,5.0,8.0,7.0,1.0,2.0
1,2019,1,10002,10028,5710,3,1,5103403,51,5,...,1.0,4.0,2.0,2.0,2.0,5.0,4.0,4.0,2.0,2.0
2,2019,1,10002,10028,5710,3,1,5103403,51,5,...,3.0,4.0,4.0,3.0,3.0,4.0,1.0,1.0,1.0,4.0
3,2019,1,10002,10028,5710,3,1,5103403,51,5,...,3.0,5.0,2.0,2.0,2.0,3.0,3.0,4.0,3.0,3.0
4,2019,1,10002,10028,5710,3,1,5103403,51,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Verifying the schema
enade.info()
dict(enade.dtypes)
enade.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 433930 entries, 0 to 433929
Columns: 137 entries, NU_ANO to QE_I68
dtypes: float64(62), int64(33), object(42)
memory usage: 453.6+ MB


(433930, 137)

### What we gonna use?
For a preliminary EDA we're gonna use the following features, mostrly the relation between NT_GER (which is the overall grade) and the other attributes.

General attributes
- CO_IES
- CO_CATEGAD
- CO_GRUPO
- CO_MODALIDADE
- CO_UF_CURSO
- NU_IDADE
- TP_SEXO
- NT_GER
- NT_FG
- NT_CE

Survey questions
- 01: Estado Civil
- 02: Cor ou Raça
- 04: Escolaridade do pai
- 05: Escolaridade da mãe
- 08: Renda familar
- 10: Situação do trabalho
- 11: Situação de bolsa
- 14: Intercambio
- 15: Cotas
- 23: Horas de estudo / semana
- 25: Motivo escolha curso
- 26: Motivo escolha IES

In [6]:
# Checking NT_GER (overall grade)
enade.NT_GER.describe()

count    390135.000000
mean         44.080390
std          14.545807
min           0.000000
25%          33.200000
50%          43.800000
75%          54.800000
max          93.000000
Name: NT_GER, dtype: float64

In [8]:
# Checking nulls
enade.NT_GER.isnull().sum() / enade.shape[0]

0.10092641670315489

In [11]:
# Checking overall grades in central-west region of BR (code 5)
enade.loc[
    enade.CO_REGIAO_CURSO == 5
].NT_GER.describe()

count    30769.000000
mean        44.604609
std         14.489081
min          0.000000
25%         33.900000
50%         44.500000
75%         55.300000
max         91.600000
Name: NT_GER, dtype: float64

In [18]:
# Now filtering grades greater than zero
enade.loc[
    (enade.CO_REGIAO_CURSO == 5) &
    (enade.NT_GER > 0)
].NT_GER.describe()

count    30739.000000
mean        44.648141
std         14.428952
min          0.900000
25%         33.900000
50%         44.500000
75%         55.300000
max         91.600000
Name: NT_GER, dtype: float64

In [19]:
# Cross table checking percentage of men and women
enade.TP_SEXO.value_counts() / enade.shape[0]

F    0.537596
M    0.462404
Name: TP_SEXO, dtype: float64

In [20]:
# Comparing regions by mean grades
enade[["NT_GER", "NT_FG", "NT_CE"]].describe()

,NT_GER,NT_FG,NT_CE
count,390135.000000,390135.000000,390135.000000
mean,44.080390,39.838931,45.477586
std,14.545807,16.241311,16.646806
min,0.000000,0.000000,0.000000
25%,33.200000,28.500000,32.900000
50%,43.800000,39.300000,45.200000
75%,54.800000,51.300000,57.800000
max,93.000000,97.800000,100.000000


In [21]:
enade.groupby("CO_REGIAO_CURSO").agg({
    "NT_GER": "mean",
    "NT_FG": "mean",
    "NT_CE": "mean"
})

,NT_GER,NT_FG,NT_CE
CO_REGIAO_CURSO,,,
1,40.282753,37.119037,41.320522
2,43.788555,40.057649,45.015455
3,44.016319,40.144793,45.290297
4,45.767208,40.087748,47.643763
5,44.604609,39.180022,46.396032


### Transforming some attributes
Some variables will be directly overwritten (even though this is not a good practice in general)

In [22]:
# Changing faculty category; creating nem column
enade["DESC_PUBLICA"] = ""

In [23]:
# Setting which IES is private
enade.loc[
    enade.CO_CATEGAD.isin([118, 120, 121, 10005, 10007, 10008, 10009, 17634]),
    "DESC_PUBLICA"
] = "Privado"

In [24]:
# Setting which IES is pulic
enade.loc[
    enade.CO_CATEGAD.isin([93, 115, 116, 10001, 10002, 10003]),
    "DESC_PUBLICA"
] = "Publica"

In [27]:
enade.DESC_PUBLICA.value_counts() / enade.shape[0]

Privado    0.760597
Publica    0.232690
           0.006713
Name: DESC_PUBLICA, dtype: float64

In [29]:
# Transforming training type (modalidade)
enade["CO_MODALIDADE"] = enade.CO_MODALIDADE.replace({
    0: "EaD",
    1: "Presencial" 
})
enade.CO_MODALIDADE.value_counts()

Presencial    410542
EaD            23388
Name: CO_MODALIDADE, dtype: int64

In [32]:
# Tranforming region
enade["CO_REGIAO_CURSO"] = enade.CO_REGIAO_CURSO.replace({
    1: "Norte",
    2: "Nordeste",
    3: "Sudeste",
    4: "Sul",
    5: "Centro-Oeste"
})

In [33]:
# Transforming skincolor
enade["DESC_COR"] = enade.QE_I02.replace({
    "A": "Branca",
    "B": "Preta",
    "C": "Amarela",
    "D": "Parda",
    "E": "Indigena",
    "F": pd.NA,
    " ": pd.NA
})
enade.DESC_COR.value_counts()

Branca      220386
Parda       135770
Preta        32869
Amarela      10093
Indigena      1300
Name: DESC_COR, dtype: int64

### Some questions

1. What is the mean grade from the northeast students?

In [36]:
enade.loc[
    enade.CO_REGIAO_CURSO == "Nordeste"    
].NT_GER.mean()

43.78855492862657

2. What is the mean specific grade of the Electrical Engineering students from RS?

In [37]:
enade.loc[
    (enade.CO_UF_CURSO == 43) &
    (enade.CO_GRUPO == 5806)
].NT_CE.mean()

34.79692671394829

3. What is the mean general formation grade of female brown students of classroom Industrial Engineerig courses, from MG?


In [39]:
enade.loc[
    (enade.CO_UF_CURSO == 31) &
    (enade.DESC_COR == "Parda") &
    (enade.CO_MODALIDADE == "Presencial") &
    (enade.CO_GRUPO == 6208) &
    (enade.TP_SEXO == "F"),
    "NT_FG"
].mean()

40.19142394822005

4. And what about the other states?

In [75]:
grades = []
for i in range(1, enade.CO_UF_CURSO.max() + 1):
    grade = enade.loc[
        (enade.CO_UF_CURSO == i) &
        (enade.DESC_COR == "Parda") &
        (enade.CO_MODALIDADE == "Presencial") &
        (enade.CO_GRUPO == 6208) &
        (enade.TP_SEXO == "F"),
        "NT_FG"
    ].mean()
    print(i, grade)
    grades.append(grade)

1 nan
2 nan
3 nan
4 nan
5 nan
6 nan
7 nan
8 nan
9 nan
10 nan
11 42.69000000000001
12 nan
13 32.611494252873555
14 nan
15 29.094594594594593
16 32.6
17 34.32
18 nan
19 nan
20 nan
21 32.52299999999999
22 40.05454545454546
23 39.16283185840707
24 44.33529411764705
25 42.49722222222221
26 38.93969465648854
27 36.72307692307693
28 37.107272727272715
29 42.811467889908265
30 nan
31 40.19142394822005
32 45.32876712328767
33 39.24093264248709
34 nan
35 38.806582278481024
36 nan
37 nan
38 nan
39 nan
40 nan
41 44.84339622641509
42 38.319999999999986
43 45.60526315789474
44 nan
45 nan
46 nan
47 nan
48 nan
49 nan
50 35.31538461538462
51 33.03589743589743
52 38.70821917808221
53 44.0875


In [83]:
# Checking those NAs
c1 = enade.loc[
    (enade.CO_UF_CURSO == 1) &
    (enade.DESC_COR == "Parda")
].shape[0]

c2 = enade.loc[
    (enade.CO_UF_CURSO == 30) &
    (enade.DESC_COR == "Parda")
].shape[0]

c3 = enade.loc[
    (enade.CO_UF_CURSO == 40) &
    (enade.DESC_COR == "Parda")
].shape[0]

print(c1, c2, c3)

# It seems that there's no declared brown people in those UF.

0 0 0
